# Tratamento dos dados
*Objetivos*: 
- Tratar o arquivo NINDINET.DBF para que seja possível a análise dos dados.
- Criar um modelo de previsão de óbitos por Leptospirose

In [101]:
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Funções Auxiliares

In [52]:
# Função para retornar os valores únicos das colunas do df
def valores_unicos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].unique()}')
        print()

# valores_unicos(df[colunas_painel])

# Função para verificar a existência de valores nulos nas colunas do df
def verifica_nulos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].isnull().sum()}')
        print()

# Função para verificar a porcentagem de nulos nas colunas do df
def porcentagem_nulos(df):
    for coluna in df.columns:
        print(f'{coluna}: {df[coluna].isnull().sum() / len(df) * 100:.2f}%')

# Função para converter a idade
def converter_idade(idade):
    faixas_etarias = {
        (0, 999): 'IGNORADO',
        (1000, 3999): '00 a 01 ano',
        (4000, 4004): '01 a 04 anos',
        (4005, 4009): '05 a 09 anos',
        (4010, 4014): '10 a 14 anos',
        (4015, 4019): '15 a 19 anos',
        (4020, 4029): '20 a 29 anos',
        (4030, 4039): '30 a 39 anos',
        (4040, 4049): '40 a 49 anos',
        (4050, 4059): '50 a 59 anos',
        (4060, 4069): '60 a 69 anos',
        (4070, 4079): '70 a 79 anos',
        (4080, 4999): 'Mais de 80 anos'
    }

    for (inicio, fim), faixa in faixas_etarias.items():
        if inicio <= idade <= fim:
            return faixa
    return 'IGNORADO'

In [53]:
# Todads as colunas do df
colunas_df = ['NU_NOTIFIC', 'TP_NOT', 'ID_AGRAVO', 'CS_SUSPEIT','IN_AIDS', 'CS_MENING',
           'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA',
           'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI', 'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 
           'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'ID_CNS_SUS', 'SG_UF', 'ID_MN_RESI', 
           'ID_RG_RESI', 'ID_DISTRIT', 'ID_BAIRRO', 'NM_BAIRRO', 'NU_CEP', 'NU_DDD_TEL', 
           'NU_TELEFON', 'CS_ZONA', 'ID_PAIS', 'NDUPLIC_N', 'IN_VINCULA', 'DT_INVEST', 
           'ID_OCUPA_N', 'CLASSI_FIN', 'CRITERIO', 'TPAUTOCTO', 'COUFINF', 'COPAISINF',
           'COMUNINF', 'CODISINF', 'CO_BAINFC', 'NOBAIINF', 'DOENCA_TRA', 'EVOLUCAO',
           'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS', 'DT_TRANSDM', 'DT_TRANSSM',
           'DT_TRANSRM', 'DT_TRANSRS', 'DT_TRANSSE', 'NU_LOTE_V', 'NU_LOTE_H', 'CS_FLXRET', 
           'FLXRECEBI', 'TPUNINOT']

# Colunas que serão analisadas
colunas_painel = ['ID_AGRAVO', 'DT_NOTIFIC', 'DT_SIN_PRI', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'DT_OBITO', 'NU_IDADE_N']

## Convertendo o arquivo .dbf
- Converter o arquivo .xls para .csv

In [54]:
# Converter de .dbf para .csv
# dbf = DBF('dbf/NINDINET.DBF')
# df = pd.DataFrame(iter(dbf))
# df.to_csv('NINDINET.csv', index=False)
# 
# df.head()

- CASO JÁ TENHA CRIADO O ARQUIVO .CSV, PODE EXECUTAR À PARTIR DAQUI
- Descomentar o código abaixo e comentar o código acima

In [55]:
# Carrega o .csv salvo
arquivo_csv = 'NINDINET.csv'

df = pd.read_csv(arquivo_csv, usecols=colunas_painel, sep=',', low_memory=False)

df.head()

,ID_AGRAVO,DT_NOTIFIC,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,DT_OBITO
0,A90,2007-02-01,2007-01-23,4029,M,6.0,4.0,2.0,NaN
1,A90,2007-02-22,2007-02-22,4018,F,9.0,9.0,9.0,NaN
2,A90,2008-02-07,2008-02-02,4002,M,6.0,4.0,10.0,NaN
3,A90,2008-01-06,2008-01-03,4025,F,5.0,4.0,6.0,NaN
4,X29,2008-01-09,2008-01-09,4047,M,6.0,4.0,9.0,NaN


## Para a análise, vamos manter apenas os casos confirmados de leptospirose

In [56]:
# Manter somente ID_AGRAVO == A279
df = df[df['ID_AGRAVO'] == 'A279']

# Remover a coluna ID_AGRAVO
df = df.drop(columns=['ID_AGRAVO'])

In [57]:
total_de_casos = len(df)

print(f'Temos um total de {total_de_casos} casos de leptospirose.')

Temos um total de 13353 casos de leptospirose.


## Tratamento de valores nulos

In [58]:
porcentagem_nulos(df)

DT_NOTIFIC: 0.00%
DT_SIN_PRI: 0.00%
NU_IDADE_N: 0.00%
CS_SEXO: 0.00%
CS_GESTANT: 0.00%
CS_RACA: 4.43%
CS_ESCOL_N: 16.45%
DT_OBITO: 95.63%


### Imputar a moda aos valores nullos das colunas CS_RACA e CS_ESCOL_N

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13353 entries, 2261 to 1661182
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  13353 non-null  object 
 1   DT_SIN_PRI  13353 non-null  object 
 2   NU_IDADE_N  13353 non-null  int64  
 3   CS_SEXO     13353 non-null  object 
 4   CS_GESTANT  13353 non-null  float64
 5   CS_RACA     12762 non-null  float64
 6   CS_ESCOL_N  11156 non-null  float64
 7   DT_OBITO    583 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 938.9+ KB


In [60]:
# Usando a moda para preencher valores nulos
df['CS_RACA'] = df['CS_RACA'].fillna(df['CS_RACA'].mode()[0])
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].fillna(df['CS_ESCOL_N'].mode()[0])


### Remover os registros das colunas com baixa quantidade de valores nulos

In [61]:
# Removendo as linhas nulas na coluna ID_MN_RESI
#df = df.dropna(subset=['ID_MN_RESI'])

### Convertendo a coluna target DT_OBITO 

In [62]:
# Transformar a coluna DT_OBITO em booleana
df['DT_OBITO'] = df['DT_OBITO'].notnull().astype(int)

### Criando a coluna NU_DIAS a partir de DT_NOTIFIC e DT_SIN_PRI

In [63]:
# Transformmar as colunas DT_NOTIFIC e DT_SIN_PRI em datetime
df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], format='%Y-%m-%d')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'], format='%Y-%m-%d')

In [64]:
# Criar uma coluna chamada NU_DIAS entre a data de notificação e a data dos primeiros sintomas
df['NU_DIAS'] = (df['DT_NOTIFIC'] - df['DT_SIN_PRI']).dt.days
df.sample(5)

,DT_NOTIFIC,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_GESTANT,CS_RACA,CS_ESCOL_N,DT_OBITO,NU_DIAS
560726,2014-03-26,2014-03-19,4038,F,5.0,4.0,2.0,0,7
605992,2024-04-11,2024-04-10,4016,F,5.0,4.0,3.0,0,1
183154,2017-01-31,2017-01-24,4045,M,6.0,4.0,3.0,0,7
506076,2017-04-24,2017-04-04,4059,M,6.0,4.0,3.0,0,20
560752,2016-04-16,2016-04-12,4008,M,6.0,4.0,1.0,0,4


In [65]:
# Remover as colunas DT_NOTIFIC e DT_SIN_PRI
df = df.drop(columns=['DT_NOTIFIC', 'DT_SIN_PRI'])

In [66]:
# Convertar a coluna NU_IDADE_N
df['FAIXA_ETARIA'] = df['NU_IDADE_N'].apply(converter_idade)

# Excluir a coluna NU_IDADE_N
df = df.drop(columns=['NU_IDADE_N'])

In [67]:
# Converter as colunas CS_SEXO, CS_GESTANT, CS_RACA, CS_ESCOL_N em variáveis dummies
df2 = pd.get_dummies(df, columns=['CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N'], drop_first=True)

# Usando Label Encoder para a variável FAIXA_ETARIA

# Definir a sequência de faixas etárias conforme a graduação que você quer seguir
faixa_etaria_ordem = [
    'IGNORADO', '00 a 01 ano', '01 a 04 anos', '05 a 09 anos', '10 a 14 anos', '15 a 19 anos', 
    '20 a 29 anos', '30 a 39 anos', '40 a 49 anos', '50 a 59 anos', '60 a 69 anos', 
    '70 a 79 anos', 'Mais de 80 anos'
]

# Criar um dicionário de mapeamento com base na ordem da lista
faixa_etaria_mapping = {faixa: idx for idx, faixa in enumerate(faixa_etaria_ordem)}

# Aplicar o mapeamento na coluna 'faixa_etaria'
df2['faixa_etaria_encoded'] = df2['FAIXA_ETARIA'].map(faixa_etaria_mapping)

# Excluindo a coluna FAIXA_ETARIA
df2 = df2.drop(columns=['FAIXA_ETARIA'])


In [68]:
df2.sample(5)

,DT_OBITO,NU_DIAS,CS_SEXO_M,CS_GESTANT_2.0,CS_GESTANT_3.0,CS_GESTANT_4.0,CS_GESTANT_5.0,CS_GESTANT_6.0,CS_GESTANT_9.0,CS_RACA_2.0,...,CS_ESCOL_N_2.0,CS_ESCOL_N_3.0,CS_ESCOL_N_4.0,CS_ESCOL_N_5.0,CS_ESCOL_N_6.0,CS_ESCOL_N_7.0,CS_ESCOL_N_8.0,CS_ESCOL_N_9.0,CS_ESCOL_N_10.0,faixa_etaria_encoded
1406932,0,128,False,False,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,False,9
940405,0,31,True,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,5
462421,0,13,True,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,7
827247,0,31,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,5
472837,1,7,True,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,4


# Aplicar Regressão Logística 

In [69]:
# Dividir o df em treino e teste, levando em consideração a coluna DT_OBITO e o desbalanceamento dos dados
X = df2.drop(columns=['DT_OBITO'])
y = df2['DT_OBITO']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [70]:
# Regressão Logística com ajuste de pesos para lidar com classes desbalanceadas
logreg_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
logreg_balanced.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [71]:
# Fazendo previsões e avaliando o modelo
y_pred_balanced = logreg_balanced.predict(X_test)
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
conf_matrix_balanced = confusion_matrix(y_test, y_pred_balanced)
class_report_balanced = classification_report(y_test, y_pred_balanced)

print(f'Acurácia {accuracy_balanced}')
print(f'Matriz de Confusão:\n {conf_matrix_balanced}')
print(class_report_balanced)


Acurácia 0.600524148259079
Matriz de Confusão:
 [[1529 1025]
 [  42   75]]
              precision    recall  f1-score   support

           0       0.97      0.60      0.74      2554
           1       0.07      0.64      0.12       117

    accuracy                           0.60      2671
   macro avg       0.52      0.62      0.43      2671
weighted avg       0.93      0.60      0.71      2671



# Aplicar Random Forest

In [72]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print(f'Acurácia {accuracy_rf}')
print(f'Matriz de Confusão:\n {conf_matrix_rf}')
print(class_report_rf)



Acurácia 0.8637214526394609
Matriz de Confusão:
 [[2290  264]
 [ 100   17]]
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      2554
           1       0.06      0.15      0.09       117

    accuracy                           0.86      2671
   macro avg       0.51      0.52      0.51      2671
weighted avg       0.92      0.86      0.89      2671



# Aplicar SMOTE para balancear as classes

In [73]:
# Aplicar o SMOTE no conjunto de treino
smote = SMOTE(k_neighbors=20, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [74]:
y_test.value_counts()

DT_OBITO
0    2554
1     117
Name: count, dtype: int64

In [75]:
# Verificar a quantidade de amostras de cada classe
y_train_smote.value_counts()


DT_OBITO
0    10216
1    10216
Name: count, dtype: int64

In [82]:
# Treinar o modelo de Regressão Logísitica após aplicar SMOTE
logreg_smote = LogisticRegression(max_iter=1000)
logreg_smote.fit(X_train_smote, y_train_smote)

# Fazer previsões no conjunto de teste
y_pred_smote = logreg_smote.predict(X_test)

# Avaliar o desempenho
accuracy_smote = accuracy_score(y_test, y_pred_smote)
conf_matrix_smote = confusion_matrix(y_test, y_pred_smote)
class_report_smote = classification_report(y_test, y_pred_smote)

# Exibir os resultados
print(f"Acurácia: {accuracy_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_smote}")
print(f"Relatório de Classificação:\n{class_report_smote}")

Acurácia: 0.899663047547735
Matriz de Confusão:
[[2381  173]
 [  95   22]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95      2554
           1       0.11      0.19      0.14       117

    accuracy                           0.90      2671
   macro avg       0.54      0.56      0.54      2671
weighted avg       0.92      0.90      0.91      2671



In [77]:
# Treinar o modelo Random Forest após o SMOTE
rf_smote = RandomForestClassifier(n_estimators=500, random_state=42, min_samples_leaf=15)
rf_smote.fit(X_train_smote, y_train_smote)

# Fazer previsões no conjunto de teste
y_pred_rf_smote = rf_smote.predict(X_test)

# Avaliar o desempenho
accuracy_rf_smote = accuracy_score(y_test, y_pred_rf_smote)
conf_matrix_rf_smote = confusion_matrix(y_test, y_pred_rf_smote)
class_report_rf_smote = classification_report(y_test, y_pred_rf_smote)



# Exibir os resultados
print(f"Acurácia: {accuracy_rf_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_rf_smote}")
print(f"Relatório de Classificação:\n{class_report_rf_smote}")

Acurácia: 0.8865593410707601
Matriz de Confusão:
[[2346  208]
 [  95   22]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2554
           1       0.10      0.19      0.13       117

    accuracy                           0.89      2671
   macro avg       0.53      0.55      0.53      2671
weighted avg       0.92      0.89      0.90      2671



# Aplicar Under Sampling



In [78]:
# Instanciar o RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)

# Aplicar o undersampling apenas no conjunto de treino
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [79]:
# Treinar o modelo de Regressão Logísitica após aplicar SMOTE
logreg_smote = LogisticRegression(max_iter=1000, solver='saga', penalty='elasticnet', l1_ratio=0.5)
logreg_smote.fit(X_train_under, y_train_under)

# Fazer previsões no conjunto de teste
y_pred_under = logreg_smote.predict(X_test)

# Avaliar o desempenho
accuracy_smote = accuracy_score(y_test, y_pred_under)
conf_matrix_smote = confusion_matrix(y_test, y_pred_under)
class_report_smote = classification_report(y_test, y_pred_under)

# Exibir os resultados
print(f"Acurácia: {accuracy_smote}")
print(f"Matriz de Confusão:\n{conf_matrix_smote}")
print(f"Relatório de Classificação:\n{class_report_smote}")

Acurácia: 0.3474354174466492
Matriz de Confusão:
[[ 828 1726]
 [  17  100]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.98      0.32      0.49      2554
           1       0.05      0.85      0.10       117

    accuracy                           0.35      2671
   macro avg       0.52      0.59      0.30      2671
weighted avg       0.94      0.35      0.47      2671



C:\Users\ricme\anaconda3\envs\ebac\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [80]:
# Instanciar o modelo Random Forest
rf = RandomForestClassifier(n_estimators=500, min_samples_leaf=15, random_state=42)

# Treinar o modelo com os dados subamostrados
rf.fit(X_train_under, y_train_under)

# Fazer previsões no conjunto de teste
y_pred = rf.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.5885436166229876
Matriz de Confusão:
[[1495 1059]
 [  40   77]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.59      0.73      2554
           1       0.07      0.66      0.12       117

    accuracy                           0.59      2671
   macro avg       0.52      0.62      0.43      2671
weighted avg       0.93      0.59      0.70      2671



In [95]:
# Usando o XGBoost
xgb_model = XGBClassifier(scale_pos_weight=12)
xgb_model.fit(X_train, y_train)

# Previsões
y_pred = xgb_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.8629726694122052
Matriz de Confusão:
[[2276  278]
 [  88   29]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.89      0.93      2554
           1       0.09      0.25      0.14       117

    accuracy                           0.86      2671
   macro avg       0.53      0.57      0.53      2671
weighted avg       0.92      0.86      0.89      2671



In [99]:
# Usando o XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train_smote, y_train_smote)

# Previsões
y_pred = xgb_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.7281916885061774
Matriz de Confusão:
[[1899  655]
 [  71   46]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.74      0.84      2554
           1       0.07      0.39      0.11       117

    accuracy                           0.73      2671
   macro avg       0.51      0.57      0.48      2671
weighted avg       0.92      0.73      0.81      2671



In [100]:
# Usando o XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train_under, y_train_under)

# Previsões
y_pred = xgb_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.6001497566454511
Matriz de Confusão:
[[1535 1019]
 [  49   68]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.60      0.74      2554
           1       0.06      0.58      0.11       117

    accuracy                           0.60      2671
   macro avg       0.52      0.59      0.43      2671
weighted avg       0.93      0.60      0.71      2671



In [103]:
ada_model = AdaBoostClassifier(n_estimators=200)
ada_model.fit(X_train, y_train)

y_pred = ada_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")


Acurácia: 0.9558217895919131
Matriz de Confusão:
[[2553    1]
 [ 117    0]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      2554
           1       0.00      0.00      0.00       117

    accuracy                           0.96      2671
   macro avg       0.48      0.50      0.49      2671
weighted avg       0.91      0.96      0.93      2671



In [106]:
ada_model = AdaBoostClassifier(n_estimators=200)
ada_model.fit(X_train_under, y_train_under)

y_pred = ada_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.6038936727817297
Matriz de Confusão:
[[1539 1015]
 [  43   74]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.97      0.60      0.74      2554
           1       0.07      0.63      0.12       117

    accuracy                           0.60      2671
   macro avg       0.52      0.62      0.43      2671
weighted avg       0.93      0.60      0.72      2671



In [108]:
ada_model = AdaBoostClassifier(n_estimators=300)
ada_model.fit(X_train_smote, y_train_smote)

y_pred = ada_model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy}")
print(f"Matriz de Confusão:\n{conf_matrix}")
print(f"Relatório de Classificação:\n{class_report}")

Acurácia: 0.8944215649569449
Matriz de Confusão:
[[2371  183]
 [  99   18]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.93      0.94      2554
           1       0.09      0.15      0.11       117

    accuracy                           0.89      2671
   macro avg       0.52      0.54      0.53      2671
weighted avg       0.92      0.89      0.91      2671

